In [1]:
#!/usr/bin/env python

def get_input(fname):
    with open(fname) as f:
        return [list(line.strip("\n")) for line in f.readlines()]

In [2]:
def print_tracks(tracks):
    print("\n".join(''.join(line) for line in tracks))

In [3]:
test_input = get_input("test.txt")

In [4]:
print_tracks(test_input)

/>-<\  
|   |  
| /<+-\
| | | v
\>+</ |
  |   ^
  \<->/


In [5]:
INTERSECTION = '+'
CURVE_CLOCKWISE = '/'
CURVE_COUNTERCLOCKWISE = '\\'
HORIZONTAL = '-'
VERTICAL = '|'

class Cart:
    UP = 0
    RIGHT = 1
    DOWN = 2
    LEFT = 3
    position = None
    direction = -1
    decision = 0
    id = 0
    def __init__(self, id, x, y, direction, decision=0):
        self.id = id
        self.position = (x, y)
        if type(direction) == int:
            self.direction = direction
        else:
            self.direction = '^>v<'.index(direction)
        self.decision = decision
    def _switch_direction_for_intersection(self):
        if self.decision == 0: # left
            self.direction = (self.direction - 1) % 4
        elif self.decision == 2:
            self.direction = (self.direction + 1) % 4
        self.decision = (self.decision + 1) % 3
    def _switch_direction_for_corner(self, curve):
        # clockwise
        move_clockwise = False
        if curve == CURVE_CLOCKWISE and self.direction in (Cart.UP, Cart.DOWN):
            move_clockwise = True
        elif curve == CURVE_COUNTERCLOCKWISE and self.direction in (Cart.RIGHT, Cart.LEFT):
            move_clockwise = True
        if move_clockwise:
            self.direction = (self.direction + 1) % 4
        else:
            self.direction = (self.direction - 1) % 4
    def _move_using_current_direction(self):
        if self.direction == Cart.RIGHT:
            self.position = (self.position[0], self.position[1] + 1)
        elif self.direction == Cart.LEFT:
            self.position = (self.position[0], self.position[1] - 1)
        elif self.direction == Cart.UP:
            self.position = (self.position[0] - 1, self.position[1])
        else:
            self.position = (self.position[0] + 1, self.position[1])
    def move(self, tracks):
        self._move_using_current_direction()
        if tracks[self.position[0]][self.position[1]] == INTERSECTION:
            self._switch_direction_for_intersection()
        elif tracks[self.position[0]][self.position[1]] in (CURVE_CLOCKWISE, CURVE_COUNTERCLOCKWISE):
            self._switch_direction_for_corner(tracks[self.position[0]][self.position[1]])
    def str_direction(self):
        directions = '^>v<'
        return directions[self.direction]
    def __repr__(self):
        directions = '^>v<'
        return "({}, {}, {})".format(self.position[0], self.position[1], self.str_direction())
    def __str__(self):
        return repr(self)

In [6]:
def process_tracks(tracks):
    carts = []
    for i in range(len(tracks)):
        for j in range(len(tracks[i])):
            if tracks[i][j] in '<>^v':
                carts.append(Cart(len(carts), i, j, tracks[i][j]))
                if tracks[i][j] in '<>':
                    tracks[i][j] = HORIZONTAL
                else:
                    tracks[i][j] = VERTICAL
    return carts, tracks

In [7]:
test_carts, test_tracks = process_tracks(get_input("test.txt"))

In [8]:
print(test_carts)

[(0, 1, >), (0, 3, <), (2, 3, <), (3, 6, v), (4, 1, >), (4, 3, <), (5, 6, ^), (6, 3, <), (6, 5, >)]


In [9]:
def print_grid(carts, tracks):
    positions = { cart.position: cart for cart in carts }
    for i in range(len(tracks)):
        for j in range(len(tracks[i])):
            if (i, j) in positions:
                print(positions[(i, j)].str_direction(), end='')
            else:
                print(tracks[i][j], end='')
        print()
    print()

In [10]:
def find_collision(tracks, carts, debug=False):
    current_positions = { cart.position: cart for cart in carts }
    loops = 0
    if debug:
        print_grid(carts, tracks)
    next_positions = current_positions
    while True:
        loops += 1
        for cart in carts:
            new_cart = Cart(cart.id, cart.position[0], cart.position[1], cart.direction, cart.decision)
            new_cart.move(tracks)
            if new_cart.position in next_positions:
                return new_cart.position
            else:
                del next_positions[cart.position]
                next_positions[new_cart.position] = new_cart
        carts = list(sorted(next_positions.values(), key=lambda c: c.position))
        current_positions = next_positions
        if debug:
            print_grid(carts, tracks)

In [11]:
find_collision(test_tracks, test_carts, debug=True)

/>-<\  
|   |  
| /<+-\
| | | v
\>+</ |
  |   ^
  \<->/



(0, 2)

In [12]:
carts, tracks = process_tracks(get_input("input.txt"))

In [13]:
# print_grid(carts, tracks)
find_collision(tracks, carts)

(3, 8)

In [14]:
def find_last_cart(tracks, carts, debug=False):
    loops = 0
    if debug:
        print_grid(carts, tracks)
    while True:
        next_positions = { cart.position: cart for cart in carts }
        carts_to_remove = set()
        loops += 1
        for cart in carts:
            if cart in carts_to_remove:
                continue
            initial_position = cart.position
            cart.move(tracks)
            if cart.position in next_positions:
                carts_to_remove.add(next_positions[cart.position])
                carts_to_remove.add(cart)
            else:
                next_positions[cart.position] = cart
            del next_positions[initial_position]
        carts = list(sorted([c for c in next_positions.values() if c not in carts_to_remove], key=lambda c: c.position))
        if len(carts) == 1:
            return carts[0]
        if debug:
            print_grid(carts, tracks)

In [15]:
test_carts, test_tracks = process_tracks(get_input("test.txt"))

In [16]:
find_last_cart(test_tracks, list(test_carts))

(4, 6, ^)

In [17]:
carts, tracks = process_tracks(get_input("input.txt"))

In [18]:
find_last_cart(tracks, list(carts))

(121, 73, <)